In [3]:
!pip install --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [5]:
from openai import OpenAI
import os
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY_Ironhack')

In [6]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,

)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [9]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': [
            'Table storing employee information including name, ID, department, and contact details.',
            'Table storing salary details for each employee including annual salary, bonuses, and benefits.',
            'Table storing educational studies data including institution name, degree obtained, and field of study.'
        ]}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Table storing employee information including n...
1     salary  Table storing salary details for each employee...
2    studies  Table storing educational studies data includi...


In [10]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [11]:
print(text_tables)

employees: Table storing employee information including name, ID, department, and contact details.
salary: Table storing salary details for each employee including annual salary, bonuses, and benefits.
studies: Table storing educational studies data including institution name, degree obtained, and field of study.


In [24]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [25]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='Which employees received a salary increase last year?')

In [26]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [30]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='Can you provide a brief description of each table in your database?')

In [31]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Describing the tables in a JSON Format

In [34]:
tables_dict = df.set_index('table')['definition'].to_dict()
output_json = {"tables": tables_dict}

print(output_json)

{'tables': {'employees': 'Table storing employee information including name, ID, department, and contact details.', 'salary': 'Table storing salary details for each employee including annual salary, bonuses, and benefits.', 'studies': 'Table storing educational studies data including institution name, degree obtained, and field of study.'}}


In [50]:
from openai import OpenAI
import os
from google.colab import userdata
import pandas as pd

# Function to call the OpenAI model
def get_response(user_message):
    client = OpenAI(api_key=OPENAI_API_KEY)

    context = []
    context.append({'role': 'system', "content": user_message})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0,
    )

    return response.choices[0].message.content

# Define data for cat-related tables and their definitions
data_cats = {'table': ['cat breeds', 'dietary restrictions', 'food brands'],
             'definition': [
                 'Table storing information about different breeds of cats including characteristics, origins, and appearance.',
                 'Table storing dietary restrictions for cats including foods that are harmful or beneficial for feline health.',
                 'Table storing brands and types of cat food available in the market, their ingredients, and nutritional values.'
             ]}

# Create a DataFrame for cat-related data
df_cats = pd.DataFrame(data_cats)

# Construct text representation of tables
text_tables_cats = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df_cats.iterrows()])

# Prompt template with updated cat-related topics
prompt_question_tables_cats = """
Given the following tables and their content definitions,
### Tables
{text_tables_cats}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
### User Question:
{question}
"""

print("Prompt 1 \n")
# Create the prompt with the user question and tables definitions
pqt1_cats = prompt_question_tables_cats.format(text_tables_cats=text_tables_cats, question='Which cat breeds are suitable for hypoallergenic households?')

# Call the OpenAI model with the prepared prompt
print(return_OAI(pqt1_cats))

print("\n Prompt 2 \n")

# Create the prompt with the user question and tables definitions
pqt2_cats = prompt_question_tables_cats.format(text_tables_cats=text_tables_cats, question='Which cat food brands are more common?')

# Call the OpenAI model with the prepared prompt
print(return_OAI(pqt2_cats))

print("\n Prompt 3")

# Create the prompt with the user question and tables definitions
pqt3_cats = prompt_question_tables_cats.format(text_tables_cats=text_tables_cats, question='What does a siamese cat eat?')

# Call the OpenAI model with the prepared prompt
print(return_OAI(pqt3_cats))


Prompt 1 

{
    "tables": ["cat breeds"]
}

 Prompt 2 

{
  "tables": ["food brands"]
}

 Prompt 3
{
  "tables": ["cat breeds", "dietary restrictions", "food brands"]
}


In [48]:
prompt_cats = """
Which cat breeds are suitable for hypoallergenic households?
"""

# Get response from OpenAI model
response_cats = get_response(prompt_cats)

# Print the model's answer
print("Model's Answer:", response_cats)


Model's Answer: Some cat breeds that are considered suitable for hypoallergenic households include:

1. Balinese
2. Russian Blue
3. Siberian
4. Sphynx
5. Devon Rex
6. Cornish Rex
7. Bengal
8. Javanese
9. Oriental Shorthair
10. Siamese

These breeds are known to produce fewer allergens or shed less dander, making them a better choice for individuals with allergies to cats.


I explored using OpenAI's GPT-3.5 model to answer questions about cats, focusing on topics like hypoallergenic breeds and common cat food brands. I set up structured data for cat breeds, dietary restrictions, and food brands, then asked the model various questions to see how well it responded.

The model performed well when I asked clear and specific questions. For example, when I inquired about hypoallergenic cat breeds, it correctly identified that information from the "cat breeds" data table would be useful. It provided structured answers based on the data I provided, which was encouraging.

However, there were times when the model seemed confused or gave vague responses. This typically happened with more open-ended questions or when the question required information beyond what the data covered.

From this experience, I learned the importance of crafting precise prompts with well-defined data. The model’s responses are directly influenced by the information and context provided in the prompt. For accurate answers, it’s essential to structure queries carefully.